In [ ]:
# Install dependencies. Need to make sure all dependencies are listed
!pip install numpy matplotlib pandas tabulate altair vega_datasets

# Abstract



# Introduction

Jupyter Notebook is a fantastic tool for data exploration, enabling analysts to write documents that contain software code, computational output, formatted text and even multimedia \cite{perkel2018jupyter}. Visualization is a big component of the data exploration process, and can be frequently found in Jupyter Notebooks: for example, a recent study of public github repositories found that *matplotlib* was the second most imported package in the notebook environment \cite{pimentel2019large}.

When datasets are too large or too complex, interactive visualization becomes a useful tool in exploratory data analysis. Interactive visualizations can enable, among many others, the display of information at multiple levels of detail, the exploration of data using coordinated views, and the dynamic change of the charts to focus on the user's interests \cite{munzner2014visualization}. While Jupyter Notebooks can contain interactive visualizations, the vast majority of charts produced in the environment are static.

In this paper, we will present three simple and powerful approaches in which data scientists can create interactive visualizations in Jupyter Notebooks: *Matplotlib Callbacks*, *Visualization Toolkits* and *Custom HTML Embedding*. The three approaches offer a number of benefits and drawbacks that need to be considered by the developer in order for them to make an informed decision about their visualization project. By the end of this paper, the reader will have a good understanding of the three methods, and will be able to select an implementation approach depending on the level of interaction, customization and data flow desired.

This paper is written entirely in a Jupyter Notebook, which can be run by the interested reader in order to interact with the visualizations and explore the source code in more detail.

# Interactive Visualization in Jupyter Notebooks

*matplotlib* \cite{hunter2007matplotlib} is the most popular general purpose visualization library for Jupyter Notebooks \cite{pimentel2019large}. This tool enables the creation static, animated, and interactive visualizations, that can be rendered directly as the output of notebook cells. The library can render visualizations in different formats, including static (raster, SVG, etc.) and interactive.  Although user interaction is supported in *matplotlib*, this is not the focus of the library: while there is support for click and keypress, interactions such drag-and-drop, tooltips, and cross-filtering, frequently supported in visualization tools \cite{munzner2014visualization}, are not directly supported.

In order to enable the creation of more interactive visualizations in Python and Jupyter Notebooks, many open source Visualization Toolkits have been developed. Among those, Perkel and others \cite{perkel2018data} highlight *Plotly* \cite{plotly2020}, *Bokeh* \cite{bokeh2020} and *Altair* \cite{vanderplas2018altair}. These libraries are built on top of web technologies, and create visualizations that can be seen in web browsers. Sintax-wise, *Plotly* and *Bokeh* are very similar to *matplotlib*. However, both libraries have been developed with a focus on user interaction, enabling the creation of web-based dashboards that combine interactive widgets and charts, and support multiple user inputs, including click, drag-and-drop, tooltips, selection, crossfilter, and bidirectional communication with Python via callbacks. *Altair* differs from the aforementioned libraries in the way visualizations are defined: it uses a declarative specification that ports  VEGA-Lite \cite{satyanarayan2016vega} grammar to Python. A wide range of interactive visualizations can be expressed using a small number of Altair primitives. However, the visualizations cannot communicate back with Python, therefore the results of user interactions cannot be used in further computations.

There might be cases when a visualization cannot be created using any off-the-shelf Python libraries. When this happens, the developer/researcher has the option to write their own visualization using a web framework, and embed this visualization in Jupyter Notebooks. This option offers the most flexibility, as the visualization can be fully customized, interactions can be scripted on demand, and even animations can be implemented. Javascript libraries such as React \cite{fedosejev2015react} and D3 \cite{bostock2011d3} can be used to facilitate the implementation of custom visualizations.

Some examples of custom Javascript visualizations in Jupyter Notebooks include libraries for scientific visualization \cite{breddels2020ipygany, breddels2020ipyvolume}, sports analytics \cite{lage2016statcast} and machine learning \cite{nori2019interpretml, ono2020pipelineprofiler}. IPyGany \cite{breddels2020ipygany} and IPyVolume \cite{breddels2020ipyvolume} enable the visualization of 3D meshes and volumes in notebooks, respectively. StatCast Dashboard \cite{lage2016statcast} supports the interactive query, filter, and visualization of spatiotemporal baseball trajectories. InterpretML \cite{nori2019interpretml} is a python package that contains a collection of algorithms for explaining and visualizing Machine Learning (ML) models, including LIME, SHAP and Partial Dependency Plots. Finally, PipelineProfiler \cite{ono2020pipelineprofiler} is a tool that enables users to explore ML pipelines produced by Automatic Machine Learning systems. 

Table 1 summarizes the different approaches to add interactive visualizations in Jupyter Notebooks. The approaches are classified in terms of interaction, type of output, level of customization, support for dashboards, and data flow. When creating a new visualization, we believe these properties should be taken into consideration.

In [ ]:
import pandas as pd
from IPython.display import display, Markdown
pd.set_option('display.notebook_repr_html', True)
librarySummary = pd.DataFrame([
    {
        'Library': 'matplotlib',
        'Interaction': 'Low',
        'Output': 'Flexible',
        'Customization': 'Low',
        'Dashboard': 'No',
        'Data Flow': 'Bidirectional'
    },
    {
        'Library': 'Plotly',
        'Interaction': 'High',
        'Output': 'HTML',
        'Customization': 'Low',
        'Dashboard': 'Yes',
        'Data Flow': 'Bidirectional'
    },
    {
        'Library': 'Bokeh',
        'Interaction': 'High',
        'Output': 'HTML',
        'Customization': 'Low',
        'Dashboard': 'Yes',
        'Data Flow': 'Bidirectional'
    },
    {
        'Library': 'Altair',
        'Interaction': 'High',
        'Output': 'HTML',
        'Customization': 'Low',
        'Dashboard': 'Yes',
        'Data Flow': 'Python => Javascript'
    },
    {
        'Library': 'Custom JS',
        'Interaction': 'High',
        'Output': 'HTML',
        'Customization': 'High',
        'Dashboard': 'Yes',
        'Data Flow': 'Bidirectional'
    }
])
display(Markdown("**Table 1**:  Summary of Interactive Visualization Approaches in Jupyter Notebook "))
display(Markdown(librarySummary.to_markdown(showindex=False)))

# Interactive Visualizations 3 Ways

In this section, we will show how to create interactive visualizations in Jupyter notebooks using three approaches discussed in the previous section: embedded *matplotlib* charts, *Altair* specifications, and custom Javascript libraries. Since the sintax of *Plotly* and *Bokeh* are so similar to *matplotlib*, we will not cover them in this paper. We encourage the interested reader to see their respective online documentations.

## Embedded Matplotlib Charts

In order to enable interactive *matplotlib* charts in the notebook environment, users need to activate this option using the *"%matplotlib notebook"* magic command [^1]. The produced charts will natively support pan and zoom operations, but can be configured to receive other types of user input, such as mouse click and key press, which can signal the run of user-defined callback functions [^2]. 

After a chart is created, for example, using *pyplot.scatter*, the user events can be captured by setting callback functions on the *canvas* using the method *mpl_connect*. Multiple events are available, including *button_press_event*, *button_release_event*, *key_press_event* and *key_release_event*.

[^1]: https://matplotlib.org/3.3.3/users/interactive.html
[^2]: https://matplotlib.org/3.3.3/users/event_handling.html

We show a minimal example below, where the visualization draws points on top of the user clicks.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots() # Creating an empty chart
def onclick(event): # Callback function
    ax.scatter([event.xdata], [event.ydata]) # Draw a point on top of the user click position.
cid = fig.canvas.mpl_connect('button_press_event', onclick) # Callback setup

As we saw, this approach can add *click* interactions to a chart with a single line of code. However, we are limited to the types of charts and interactions supported by matplotlib. When these options are not enough, the developer might need to consider other libraries, such as *Altair*, or creating their own visualization in Javascript. 

## Altair Specification

Altair enables the creation of interactive visualizations by using a pythonic port of the Vega-Lite specification \cite{vanderplas2018altair}. Altair uses a declarative visualization paradigm: instead of telling the library every step of how to draw a chart, the programmer specifies the data and the visual encodings, and the library takes care of the rest.

In order to create a chart, the developer needs to have a *Pandas DataFrame* containing the data to be visualized. An *Altair.Chart* object needs to be created, with the corresponding *DataFrame* passed as a parameter. Next, an *encoding* and a *mark* needs to be selected. *Encodings* tell *Altair* how the *DataFrame* columns should be mapped to visual attributes. Meanwhile, *marks* specify how the attributes should be represented on the plot (for example, as a circle, line, area chart, etc).

We show a basic example of an Altair scatter plot with the Iris dataset. The dataset contains information regarding 150 Iris flowers, with measurements of length and width of the plant, as well as the flower species. In the chart below, the scatterplot shows petalLength and petalWidth in the cartesian plane. Color is used to show the flower species. Finally, data points can be hovered to show additional information as a tooltip (notice that this was not possible in *matplotlib*). In the code below, *mark_circle* is used to indicate the type of chart desired (scatter plot with circles) and the *encode* function specify the chart encoding, in this case, what columns are mapped to the *x* and *y* positions, *color* of the circle, and *tooltip* on hover.

In [ ]:
import altair as alt
from vega_datasets import data

data = data.iris()

alt.Chart(data).mark_circle().encode(
    x='petalLength',
    y='petalWidth',
    color='species',
    tooltip=['sepalLength', 'sepalWidth', 'petalLength', 'petalWidth', 'species']
)

![](Images/AltairIris.png)

For more complex examples, please see the Altair documentation. There are many chart possibilities, and charts can be combined to create interactive dashboards. As an example of dashboard, this page[^3] shows how to create a chart with three distinct views and crossfilter capabilities.

[^3]: https://altair-viz.github.io/gallery/interactive_layered_crossfilter.html

## Custom Javascript Visualizations

Displaying custom Javascript visualizations in Jupyter Notebook can be done in a few lines of code. 
The package *Ipython.display*[^1] contains the function *display* and the class *HTML*, which together can embed any HTML code in notebook cells. The HTML may contain both CSS and Javascript, which affords flexible, interactive and customizable visualizations to be created.

[^1]: https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

In order to embed the the visualization in a cell, one needs to create a string variable containing all the HTML, Javascript and CSS code needed for the visualization. Since writing everything in a Jupyter cell can be too cumbersome, one can write the visualization in a code editor, then load the document in python and display it. Javascript Bundlers, such as Webpack \cite{webpack_webpack_2020}, can convert multiple HTML, Javascript and CSS files into a single file, facilitating this process.

In the following, we show a minimal example of HTML embedding in a jupyter cell:

In [ ]:
from IPython.display import display, HTML
html_string = """<button onclick="alert('Hello World')">Hello World</button>"""
display(HTML(html_string))

Two popular frameworks for writing Javascript visualizations are D3 \cite{bostock2011d3} and React \cite{fedosejev2015react}. Both frameworks manipulate the HTML page based on the data, which facilitates the design of new visualizations. Integrating these third party frameworks in custom Jupyter Notebook visualizations can be accomplished in two ways: 1) Using Webpack \cite{webpack2020} to bundle all the dependencies and visualization source code together in  a new library file. And 2) Importing the framework directly on a notebook cell using the *Require* command. 

The first approach has a higher setup cost, as it requires setting up a new NPM project and configuring Webpack to bundle the code and dependencies into a single javascript file. However, it allows the visualization to be divided in multiple files, which often results in a more organized code. For detailed instructions on how to compile a javascript library using Webpack, please see the documentation page[^2].

[^2]: https://webpack.js.org/guides/author-libraries/

Once the code is bundled into a single file, it can be used in Jupyter notebooks by reading the  file as a text document, and displaying it using the *display(HTML(source))* command.

Data can be passed from Python to javascript by using JSON strings. First, the data is converted to json. Then, the string can be sent to a variable in javascript and used normally.

The following code snippet shows how to embed a custom library that enables the display and edit of baseball play trajectories. The trajectory data comes from \cite{ono2019historytracker}

In [ ]:
import pandas as pd

with open("./BaseballVisualizer/build/baseballvisualizer.js", "r") as f:
    baseball_visualizer_bundle = f.read()

play = pd.read_csv("./play_annotated.csv", sep=";")
    
html = """
<html>
<body>
<div id="container"/>

<script type="application/javascript">
{baseball_visualizer_bundle}
</script>

<script> 
     baseballvisualizer.renderBaseballAnnotator("#container", {data});
</script>

</body>
</html>
""".format(baseball_visualizer_bundle=baseball_visualizer_bundle, data={'tracking': play.to_json(orient="records")})

display(HTML(html))

In [ ]:
received_trajectory = []
def receive_trajectory(comm, open_msg):
    # comm is the kernel Comm instance
    # Register handler for later messages
    @comm.on_msg
    def _recv(msg):
        global received_trajectory 
        # Use msg['content']['data'] for the data in the message
        received_trajectory = msg['content']['data']['trajectory']
        print(received_trajectory)
        comm.send({'received': True})

get_ipython().kernel.comm_manager.register_target('submit_trajectory', receive_trajectory)

In [ ]:
pd.DataFrame(received_trajectory)